In [1]:
import os
import json
import pandas as pd


# Excel 파일들이 위치한 디렉토리 경로
directory_path = 'C:/Users/ch.kang/OneDrive - 클리 주식회사/문서 - 클리주식회사/000_Routine_마이세컨플레이스/02. IT Platform&Data/01_콘텐츠성과측정/01_DailyRawdata'

# ga로 시작하는 모든 Excel 파일 목록을 오름차순으로 정렬
excel_files = sorted([file for file in os.listdir(directory_path) if file.startswith('ga') and file.endswith('.xlsx')])

# 각 파일을 데이터프레임으로 불러와서 합치기
all_data_frames = []  # 불러온 데이터프레임을 저장할 리스트

for file in excel_files:
    file_path = os.path.join(directory_path, file)
    # 모든 컬럼의 데이터 타입을 문자열로 지정하여 Excel 파일 불러오기
    df = pd.read_excel(file_path, dtype=str)
    all_data_frames.append(df)

# 모든 데이터프레임을 단일 데이터프레임으로 합치기
combined_df = pd.concat(all_data_frames, ignore_index=True)

# 여기서 combined_df를 사용하여 추가 작업 수행


In [2]:
df = combined_df

In [3]:
print(len(df))
print(df.columns)

170824
Index(['아이디', '시간', '이벤트명', '페이지제목', '정규화링크', '원래링크', '소스', '매체', '캠페인', '컨텐츠',
       '검색어', 'engagement_time', '체류시간', '전환여부', '익명아이디'],
      dtype='object')


In [4]:
class UniqueIDGenerator:
    def __init__(self, mapping_file):
        self.mapping_file = mapping_file
        self.id_map = self.load_mapping()
        self.unique_id_counter = self.get_unique_id_counter()

    def load_mapping(self):
        if os.path.exists(self.mapping_file):
            try:
                with open(self.mapping_file, 'r') as f:
                    return json.load(f)
            except json.JSONDecodeError:
                print(f"Warning: Invalid JSON format in {self.mapping_file}. Creating new mapping.")
        return {}

    def get_unique_id_counter(self):
        max_id = 0
        for id_str in self.id_map.values():
            if id_str.startswith('g-'):
                try:
                    id_num = int(id_str.split('-')[1])
                    max_id = max(max_id, id_num)
                except ValueError:
                    pass
        return max_id + 1

    def save_mapping(self):
        with open(self.mapping_file, 'w') as f:
            json.dump(self.id_map, f, indent=4)

    def generate_unique_id(self, username, anonymous_id):
        if pd.isnull(anonymous_id):
            if username in self.id_map:
                return self.id_map[username]
            else:
                unique_id = f"g-{self.unique_id_counter:06d}"
                self.unique_id_counter += 1
                self.id_map[username] = unique_id
                self.id_map[unique_id] = None
                return unique_id
        else:
            if username in self.id_map:
                return self.id_map[username]
            elif anonymous_id in self.id_map.values():
                for key, value in self.id_map.items():
                    if value == anonymous_id:
                        return key
            else:
                unique_id = f"g-{self.unique_id_counter:06d}"
                self.unique_id_counter += 1
                self.id_map[username] = unique_id
                self.id_map[unique_id] = anonymous_id
                return unique_id

# JSON 파일 경로
mapping_file = r'C:\Users\ch.kang\OneDrive - 클리 주식회사\문서 - 클리주식회사\000_Routine_마이세컨플레이스\02. IT Platform&Data\01_콘텐츠성과측정\02_ConvertedData\id_mapping.json'

# UniqueIDGenerator 인스턴스 생성
generator = UniqueIDGenerator(mapping_file)

# 고유 아이디 생성 및 데이터프레임에 추가
df['고유아이디'] = df.apply(lambda row: generator.generate_unique_id(row['아이디'], row['익명아이디']), axis=1)

# 결과를 JSON 파일로 저장
generator.save_mapping()

# 결과 확인
print(df[['고유아이디', '아이디', '익명아이디']])


           고유아이디                    아이디                                 익명아이디
0       g-000001  1038925075.1705314572                                   NaN
1       g-000001  1038925075.1705314572                                   NaN
2       g-000002  1079045696.1705309555                                   NaN
3       g-000002  1079045696.1705309555                                   NaN
4       g-000002  1079045696.1705309555                                   NaN
...          ...                    ...                                   ...
170819  g-030300   993475916.1713917639  6b2a6f99-a3ee-4ef3-8cbc-3e87d1fb33bd
170820  g-030300   993475916.1713917639  6b2a6f99-a3ee-4ef3-8cbc-3e87d1fb33bd
170821  g-030300   993475916.1713917639  6b2a6f99-a3ee-4ef3-8cbc-3e87d1fb33bd
170822  g-030300   993475916.1713917639  6b2a6f99-a3ee-4ef3-8cbc-3e87d1fb33bd
170823  g-030300   993475916.1713917639  6b2a6f99-a3ee-4ef3-8cbc-3e87d1fb33bd

[170824 rows x 3 columns]


In [5]:
filter_values = ['g-021091', 'g-000193', 'g-001151', 'g-001540', 'g-000293', 'g-002308']
value_name_mapping = {
    'g-021091': '김민좌',
    'g-000193': '최선하',
    'g-001151': '전진',
    'g-001540': '이예슬',
    'g-000293': '양지은',
    'g-002308': '소주희'
}
filtered_df = df[df['고유아이디'].isin(filter_values)]
# 새로운 열 추가
filtered_df['이름'] = filtered_df['고유아이디'].map(value_name_mapping)
# filtered_df.to_excel("투어신청자_0406-0414.xlsx", index=False)


C:\Users\ch.kang\AppData\Local\Temp\ipykernel_15908\787139387.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['이름'] = filtered_df['고유아이디'].map(value_name_mapping)
